# Modeling of a Supercritical CO2 Joule Cycle

In recent years, the search for efficient and sustainable energy solutions has led to significant advances in power generation technologies. Among these innovations, the Supercritical CO2 Joule Cycle stands out as a promising candidate for improving the efficiency and economics of thermal power plants.

Supercritical CO2 (sCO2) Brayton-cycle energy conversion systems transform heat energy into electrical energy using sCO2 rather than steam in the Rankine cycle systems commonly used in today's CSP, coal, nuclear, and combined-cycle gas plants. Studies indicate that the closed-loop sCO2 cycle can provide higher cycle efficiency versus superheated or supercritical steam cycles at temperatures relevant for CSP applications. Brayton-cycle systems using sCO2 are more compact, have lower thermal mass, and simpler power blocks compared to Rankine cycles. However, this cycle needs high temperatures when compared to the actual temperatures below 600°C. There are several potential technology pathways including particle, advanced molten-salt, and gas-phase HTFs, which are being investigates to reach higher temperatures. [Concentrating Solar Power Gen3 Demonstration Roadmap](https://www.nrel.gov/docs/fy17osti/67464.pdf)

The exercise is heavily based on the paper:
> Penkuhn M, Tsatsaronis G. Exergoeconomic analyses of different sCO2 cycle configurations. InProceedings of the 6th International Symposium—Supercritical CO2 Power Cycles, Pittsburgh, PA, USA 2018 Mar (pp. 27-29). [https://sco2symposium.com/papers2018/cycles/052_Paper.pdf](https://sco2symposium.com/papers2018/cycles/052_Paper.pdf)

And the code by Francesco Witte:
<https://github.com/fwitte/sCO2_exergy> archived with DOI: [10.5281/zenodo.4751796](https://zenodo.org/record/4751796)
## Authors

The authors of this exercise are:

- Amedeo Ceruti
- Urbano Tataranni
- Hassan Hussein
- Jihad Jundi

## Defining the Supercritical CO2 Joule Cycle

The code for the power plant simulation is performed in a very similar manner to that performed for exercise 03. The cycle consists of several key components:
- **Compressors (CP1 and CP2)**: These components compress the CO2 to higher pressures, crucial for maintaining cycle efficiency.
- **Heat Exchangers (Recuperators)**: Recuperators facilitate heat exchange between exhaust streams, optimizing thermal efficiency.
- **Turbine**: The turbine expands CO2 to generate mechanical power, which is then converted into electrical energy.
- **Heat Exchangers (Heater and Cooler)**: These components facilitate heat transfer to and from external sources, crucial for maintaining working fluid temperatures within operational limits.

![Representation of the cycle under examination](figs/CO2.svg)

In [1]:
from tespy.networks import Network
from tespy.components import (
    Sink, Source, Turbine, SimpleHeatExchanger, Merge, Splitter,
    HeatExchanger, CycleCloser, Compressor, PowerBus, PowerSink, Generator, Motor)
from tespy.connections import PowerConnection, Connection, Bus

In [2]:
# setting up network
jouleCycle = Network(fluids=['CO2'])
jouleCycle.units.set_defaults(
    temperature='degC', pressure='bar', enthalpy='kJ / kg', mass_flow='kg / s',
    entropy="kJ / kgK")


The needed components are defined for the thermal cycle.

- **Source and Sink**: `water_in` represents the water source in the cycle and `water_out` represents the water sink in the cycle.

- **CycleCloser**: `closer` is the component that closes the cycle, ensuring mass balance between inlet and outlet.

- **Compressor**: `cp1` and `cp2` are the two compressors in the cycle (Compressor 1 and Compressor 2), both belonging to the functional group 'CMP'.

- **HeatExchanger**: `rec1` and `rec2`are the two heat exchangers in the cycle (Recuperator 1 and Recuperator 2), both belonging to the functional group 'REC'.

- **SimpleHeatExchanger**: `cooler` as a simple heat exchanger for cooling water and `heater` as a simple heat exchanger for heating the fluid in the cycle.

- **Turbine**: `turb` is the turbine in the cycle, which converts fluid energy into mechanical energy.

- **Splitter and Merge**: `sp1` is the splitter in the cycle, part of the functional group 'REC' and `m1` is the merge in the cycle, part of the functional group 'REC'.


In [3]:
# components definition
water_in = Source('Water source')
water_out = Sink('Water sink')

closer = CycleCloser('Cycle closer')

cp1 = Compressor('Compressor 1', fkt_group='CMP')
cp2 = Compressor('Compressor 2', fkt_group='CMP')

rec1 = HeatExchanger('Recuperator 1', fkt_group='REC')
rec2 = HeatExchanger('Recuperator 2', fkt_group='REC')

cooler = SimpleHeatExchanger('Water cooler')
heater = SimpleHeatExchanger('Heater')

turb = Turbine('Turbine')

sp1 = Splitter('Splitter 1', fkt_group='REC')
m1 = Merge('Merge 1', fkt_group='REC')

Each connection is defined by specifying the components and their respective ports that are being connected. The connections are then added to the cycle network.

- `c1` connects the outlet of the water cooler (`cooler`) to the inlet of Compressor 1 (`cp1`).
- `c2` connects the outlet of Compressor 1 (`cp1`) to the inlet of Recuperator 1 (`rec1`).
- `c3` connects the outlet of Recuperator 2 (`rec2`) to the inlet of the heater (`heater`).
- `c0` connects the outlet of the heater (`heater`) to the inlet of the cycle closer (`closer`).
- `c4` connects the outlet of the cycle closer (`closer`) to the inlet of the turbine (`turb`).
- `c5` connects the outlet of the turbine (`turb`) to the inlet of Recuperator 2 (`rec2`).
- `c6` connects the outlet of Splitter 1 (`sp1`) to the inlet of the water cooler (`cooler`).
- `c10` connects the outlet of Splitter 1 (`sp1`) to the inlet of Compressor 2 (`cp2`).
- `c11` connects the outlet of Compressor 2 (`cp2`) to the inlet of Merge 1 (`m1`).
- `c12` connects the outlet of Recuperator 1 (`rec1`) to the inlet of Merge 1 (`m1`).
- `c13` connects the outlet of Merge 1 (`m1`) to the inlet of Recuperator 2 (`rec2`).
- `c14` connects the outlet of Recuperator 2 (`rec2`) to the inlet of Recuperator 1 (`rec1`).
- `c15` connects the outlet of Recuperator 1 (`rec1`) to the inlet of Splitter 1 (`sp1`).

In [4]:
# connections definition power cycle
c1 = Connection(cooler, 'out1', cp1, 'in1', label='1')
c2 = Connection(cp1, 'out1', rec1, 'in2', label='2')
c3 = Connection(rec2, 'out2', heater, 'in1', label='3')

c0 = Connection(heater, 'out1', closer, 'in1', label='0')
c4 = Connection(closer, 'out1', turb, 'in1', label='4')
c5 = Connection(turb, 'out1', rec2, 'in1', label='5')
c6 = Connection(sp1, 'out1', cooler, 'in1', label='6')

c10 = Connection(sp1, 'out2', cp2, 'in1', label='10')
c11 = Connection(cp2, 'out1', m1, 'in2', label='11')
c12 = Connection(rec1, 'out2', m1, 'in1', label='12')
c13 = Connection(m1, 'out1', rec2, 'in2', label='13')

c14 = Connection(rec2, 'out1', rec1, 'in1', label='14')
c15 = Connection(rec1, 'out1', sp1, 'in1', label='15')

# add connections to network
jouleCycle.add_conns(c0, c1, c2, c3, c4, c5, c6, c10, c11, c12, c13, c14, c15)

In [5]:
power_bus = PowerBus('total output power', num_in=1, num_out=3)

# Create efficiency components
turb_generator = Generator('turbine generator')
cp1_motor = Motor('compressor 1 motor')
cp2_motor = Motor('compressor 2 motor')
grid = PowerSink('power output')

# turbine generates power and feeds it into the powerbus
e1 = PowerConnection(turb, 'power', turb_generator, 'power_in', label='e1')
e2 = PowerConnection(turb_generator, 'power_out', power_bus, 'power_in1', label='e2')

# Compressors 1 and 2 consume power from the powerbus
e3 = PowerConnection(power_bus, 'power_out1', cp1_motor, 'power_in', label='e3')
e4 = PowerConnection(cp1_motor, 'power_out', cp1, 'power', label='e4')
e5 = PowerConnection(power_bus, 'power_out2', cp2_motor, 'power_in', label='e5')
e6 = PowerConnection(cp2_motor, 'power_out', cp2, 'power', label='e6')

# Grid export connection that gives us the total power output
e_grid = PowerConnection(power_bus, 'power_out3', grid, 'power', label='E')
e_grid.set_attr(E=None)  # track power output to the grid

# Add connections to network
jouleCycle.add_conns(e1, e2, e3, e4, e5, e6, e_grid)

# Set efficiencies
turb_generator.set_attr(eta=0.99 * 0.99)  # 98.01% efficiency
cp1_motor.set_attr(eta=0.98 * 0.97)       # 95.06% efficiency  
cp2_motor.set_attr(eta=0.98 * 0.97)       # 95.06% efficiency

In [6]:
heat_bus = Bus('heat input')
heat_bus.add_comps({'comp': heater, 'base': 'bus'})

In [7]:
jouleCycle.add_busses(heat_bus)

Setting the attributes for the connections and the components. These parameters include temperatures, pressures, mass flow rates, and efficiencies. It was created in such a way as to ensure stable operation over different TIT ranges by parametrically adopting the pressure losses and temperature differences at the heat exchangers. All data are referred to [Exergoeconomic analyses of different sCO2 cycle configurations](https://sco2symposium.com/papers2018/cycles/052_Paper.pdf).

- `c1` is set with a temperature (`T`) of 35°C, a pressure (`p`) of 75 bar, and a fluid composition of 100% CO2.
- `c4` is set with a temperature (`T`) of 384°C, a pressure (`p`) of 258 bar, and a mass flow rate (`m`) of 54 kg/s.
- `c5` is set with a pressure (`p`) of 78 bar.
- `rec1` (Recuperator 1) is set with a pressure ratio (`pr1` and `pr2`) of 0.977 and 0.998 respectively, a lower terminal temperature difference (`ttd_l`) of 10°C, and an upper terminal temperature difference (`ttd_u`) of 25°C.
- `rec2` (Recuperator 2) is set with a pressure ratio (`pr1` and `pr2`) oof 0.987 and 0.997 respectively and a lower terminal temperature difference (`ttd_l`) of 10°C.
- `heater` is set with a pressure ratio (`pr`) of 0.973.

And for the component parameters:

- `turb` (Turbine) is set with an isentropic efficiency (`eta_s`) of 0.90.
- `cp1` (Compressor 1) is set with an isentropic efficiency (`eta_s`) of 0.85.
- `cp2` (Compressor 2) is set with an isentropic efficiency (`eta_s`) of 0.85.


In [27]:
# connection parameters
c1.set_attr(T=35, p=75, fluid={'CO2': 1})
c4.set_attr(T=400, p=258, m=54)
c5.set_attr(p=78)
rec1.set_attr(pr1=0.977, pr2=0.998, ttd_l=10, ttd_u=25)
rec2.set_attr(pr1=0.987, pr2=0.997, ttd_l=10)
heater.set_attr(pr=0.973)

# component parameters
turb.set_attr(eta_s=0.9)
cp1.set_attr(eta_s=0.85)
cp2.set_attr(eta_s=0.85)

## Results

In [28]:
# solve final state
jouleCycle.set_attr(iterinfo=False)
jouleCycle.solve(mode='design')

# print results to prompt and generate model documentation
jouleCycle.print_results()


##### RESULTS (Compressor) #####
+--------------+----------+----------+-----------+----------+
|              |        P |       pr |        dp |    eta_s |
|--------------+----------+----------+-----------+----------|
| Compressor 1 | 2.47e+06 | 3.55e+00 | -1.91e+02 | 8.50e-01 |
| Compressor 2 | 1.02e+06 | 3.54e+00 | -1.91e+02 | 8.50e-01 |
+--------------+----------+----------+-----------+----------+
##### RESULTS (CycleCloser) #####
+--------------+------------------+-------------------+
|              |   mass_deviation |   fluid_deviation |
|--------------+------------------+-------------------|
| Cycle closer |         0.00e+00 |          0.00e+00 |
+--------------+------------------+-------------------+
##### RESULTS (SimpleHeatExchanger) #####
+--------------+-----------+----------+----------+----------+
|              |         Q |       pr |       dp |     zeta |
|--------------+-----------+----------+----------+----------|
| Heater       |  1.07e+07 | 9.73e-01 | 7.16e+00 | 7

## 1. Calculate the idealized total efficiency of the cycle

Neglect the efficiecy losses of the motors and generators.

In [ ]:
W_net = ...
Q_in = ...
efficiency = ...
print(f"Efficiency for a TIT equal to 384°C as in Andasol 2: {round(efficiency * 100, 2)} %")

## 2. Calculate the efficiency including power component efficiency

Hint: use the `e_grid.E.val` value

In [ ]:
efficiency2 = ...
print(f"Efficiency for a TIT equal to 384°C as in Andasol 2: {round(efficiency2 * 100, 2)} %")

## 3. What is the theoretical obtainable efficiency for CSP sCO2 cycles?

Use the following conference paper: <https://doi.org/10.1016/j.egypro.2014.03.128>

## 4. How is the Joule Cycle called in the US?

## 5. Recalculate the conditions for a molten salt power plant

HInt: just modify the `c4` connection temperature attribute such as:

``c4.set_attr(T=XXX, p=258, m=54)``

In [ ]:
c4.set_attr(T=..., p=258, m=54)
jouleCycle.solve(mode='design')
# jouleCycle.print_results()

In [ ]:
print(f"Efficiency molten salt: {round(... * 100, 2)} %")

## 6. What receiver configurations are mentioned in the conference paper to push the temperature higher?

Target temperatures by advanced receiver configurations 650 C by direct use of CO2 as working fluid and receiver heat transfer fluid.

In [33]:
c4.set_attr(T=650, p=258, m=54)
jouleCycle.solve(mode='design')
# jouleCycle.print_results()

In [ ]:
print(f"Efficiency direct use: {round(... * 100, 2)} %")